In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from UV_views import *
from Y_views import *
from efficiency import *
%matplotlib inline

In [2]:
all_hits = pd.read_csv('Data/strawtubesPoint_decoded_hits.csv', delimiter=',')
all_hits['Index'] = all_hits.index
all_hits = all_hits[['Index', 'X', 'Y', 'Z', 'TrackID', 'Time', 'StatNb', 'ViewNb', 'PlaneNb', 'LayerNb', 'StrawNb', 'dist2Wire', 'event', 'k']]
all_hits.head()

,Index,X,Y,Z,TrackID,Time,StatNb,ViewNb,PlaneNb,LayerNb,StrawNb,dist2Wire,event,k
0,0,8.883409,-61.202305,-1974.859985,3,232.044861,5,0,0,0,319,0.274822,0,0
1,1,8.858191,-61.225544,-1973.742188,3,232.074081,5,0,0,1,320,0.582455,0,1
2,2,8.824563,-61.256523,-1972.251953,3,232.118286,5,0,1,0,319,0.111252,0,2
3,3,8.632817,-61.432972,-1963.756592,3,232.406479,5,1,0,1,320,0.164412,0,3
4,4,8.573978,-61.487133,-1961.149292,3,232.488541,5,1,1,1,320,0.226827,0,4


In [3]:
def combine_1_2(tracks_yz, tracks_xz):
    pairs = {}
    s = tracks_xz.keys()
    for i in s:
        for j in s:
            if (i != j) and (i not in pairs) and (j not in pairs):
                z_x = (tracks_xz[i][1] - tracks_xz[j][1]) / (tracks_xz[j][0] - tracks_xz[i][0])
                if (z_x > -2000 and z_x < 2598):
                    if (i / 10000 == j / 10000):
                        pairs[i] = j
                        pairs[j] = i
                    else:
                        z_y = (tracks_yz[i / 10000][1] - tracks_yz[j / 10000][1]) / (tracks_yz[j / 10000][0] - tracks_yz[i / 10000][0])
                        if (z_y > z_x + 100 and z_y < z_x - 100):
                            pairs[i] = j
                            pairs[j] = i
    return pairs

In [4]:
def matching(tracks_yz_before, tracks_xz_before, tracks_yz_after, tracks_xz_after, pairs):
    match = {}
    for i in pairs:
        for j in tracks_xz_after:
            if (i not in match.keys() and j not in match.values()):
                x_before = tracks_xz_before[i][0] * 3000 + tracks_xz_before[i][1]
                x_after = tracks_xz_after[j][0] * 3000 + tracks_xz_after[j][1]
                if (x_before > x_after - 20 and x_before < x_after + 20):
                    y_before = tracks_yz_before[i / 10000][0] * 3000 + tracks_yz_before[i / 10000][1]
                    y_after = tracks_yz_after[j / 10000][0] * 3000 + tracks_yz_after[j / 10000][1]
                    if (y_before < y_after + 2 and y_before > y_after - 2):
                        match[i] = j
    return match

In [5]:
for i in range(1, 3000):
    event = all_hits[all_hits['event'] == i]
    tracks_yz_1_2, linking_table_yz_1_2 = loop_yz(event, 7, 0.85, 0)
    if len(tracks_yz_1_2) < 2:
        continue
    #print i
    tracks_xz_1_2, linking_table_xz_1_2, Xs = loop_xz(event, tracks_yz_1_2, linking_table_yz_1_2, 6, 7, 0)
    if len(tracks_xz_1_2) < 2:
        continue
    #print i
    tracks_yz_3_4, linking_table_yz_3_4 = loop_yz(event, 7, 0.85, 1)
    if len(tracks_yz_3_4) < 2:
        continue
    #print i
    tracks_xz_3_4, linking_table_xz_3_4, Xs = loop_xz(event, tracks_yz_3_4, linking_table_yz_3_4, 6, 7, 1)
    if len(tracks_xz_3_4) < 2:
        continue
    print i
    pairs = combine_1_2(tracks_yz_1_2, tracks_xz_1_2)
    #print pairs
    if len(pairs) < 1:
        continue
    print "yest' pairs"
    match = matching(tracks_yz_1_2, tracks_xz_1_2, tracks_yz_3_4, tracks_xz_3_4, pairs)
    if len(match) > 0:
        print "index:" + str(i)

2
10
15
23
25
27
32
39
yest' pairs
42
48
50
53
62
74
77
89
94
97
102
yest' pairs
106
121
122
125
126
129
131
134
135
yest' pairs
157
159
161
174
179
181
183
184
185
189
195
211
215
224
225
yest' pairs
index:225
228
243
249
253
255
257
276
289
292
294
295
299
331
341
346
353
356
365
366
369
370
371
377
380
381
391
392
406
410
411
412
417
425
428
445
449
455
463
468
479
484
490
494
500
502
513
516
518
532
542
547
549
553
558
565
580
595
600
608
612
614
616
622
623
625
631
640
642
644
648
651
654
655
658
662
669
677
684
686
692
694
707
711
712
719
720
723
735
736
741
747
756
759
761
767
768
777
781
782
783
785
791
793
803
806
809
813
817
824
828
839
842
845
851
855
856
861
863
868
876
878
883
885
893
897
898
901
902
906
917
923
927
928
yest' pairs
931
yest' pairs
933
935
940
942
951
953
957
963
968
970
971
975
978
986
987
1001
1006
1008
1009
1011
1014
1017
1018
1021
1022
1023
1036
1042
1057
1061
1084
1086
1090
1111
1116
1119
1120
1129
1136
1138
1139
1153
1156
1157
1180
yest' pairs
1190
11

/usr/local/lib/python2.7/site-packages/pandas/core/indexing.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python2.7/site-packages/pandas/core/indexing.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
